In [1]:
from sqlalchemy import extract, desc
import datetime
import time
import MetaTrader5 as mt5
import pandas as pd

import options_estrategies as opt_est
from options_estrategies import Option_Due, InformationType
import models as model

In [2]:
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

estrategies = opt_est.options_estrategies(model, model.PETR4, model.PETR4_OPTIONS)

In [3]:
def convert_dict_from_update_ticks_to_dataframe(tick_data):
    df_tick_data = pd.DataFrame.from_dict(tick_data)
    df_tick_data = df_tick_data.drop(columns=['_sa_instance_state','timestamp_option','days_to_due_date'])
    df_tick_data['ratio'] = df_tick_data['last_tick'] - df_tick_data['last_tick'].shift(1)
    df_tick_data = df_tick_data[['option_name', 'strike', 'bid', 'ask', 'last_tick','ratio','stock_price','deal_type_zone','updated_at']]      
    return df_tick_data


In [42]:
updated_ticks = estrategies.update_quotes(just_last_update=False,mode=InformationType.Offline)
ticks_to_process = updated_ticks[0]

options = []
for i in ticks_to_process:
    options.append(i.__dict__)

dataframe_historical_ticks = convert_dict_from_update_ticks_to_dataframe(options)
dataframe_historical_ticks

,option_name,strike,bid,ask,last_tick,ratio,stock_price,deal_type_zone,updated_at
0,PETRH269,26.21,2.46,2.53,2.38,NaN,28.09,ITM,2021-07-14 10:09:44.647
1,PETRH254,26.46,2.28,2.34,2.19,-0.19,28.09,ITM,2021-07-14 10:09:44.647
2,PETRH274,26.71,2.11,2.16,2.02,-0.17,28.09,ITM,2021-07-14 10:09:44.647
3,PETRH276,26.96,1.92,1.98,1.83,-0.19,28.09,ITM,2021-07-14 10:09:44.647
4,PETRH261,27.21,1.76,1.82,1.81,-0.02,28.09,ITM,2021-07-14 10:09:44.647
...,...,...,...,...,...,...,...,...,...
34570,PETRH243,28.96,0.63,0.65,0.65,-0.08,27.49,OTM,2021-07-15 11:12:16.510
34571,PETRH299,29.21,0.56,0.57,0.56,-0.09,27.49,OTM,2021-07-15 11:12:16.510
34572,PETRH251,29.46,0.49,0.50,0.51,-0.05,27.49,OTM,2021-07-15 11:12:16.510
34573,PETRH304,29.71,0.43,0.44,0.43,-0.08,27.49,OTM,2021-07-15 11:12:16.510


In [45]:
ratio_statistic_finder = dataframe_historical_ticks[['updated_at','option_name','ratio']]
ratio_statistic_finder = ratio_statistic_finder.set_index('updated_at')
ratio_statistic_finder = ratio_statistic_finder.between_time('10:10', '17:00')
ratio_statistic_finder = ratio_statistic_finder.reset_index()
ratio_statistic_finder = ratio_statistic_finder.pivot(index='updated_at', columns='option_name', values='ratio')
ratio_statistic_finder = ratio_statistic_finder.dropna(axis=1)
ratio_statistic_finder

ratio_statistic_data = {}
for (column, value) in ratio_statistic_finder.iteritems():
    ratio_statistic_data.update({ column: { "Mean": ratio_statistic_finder[column].mean(), "StdDev": ratio_statistic_finder[column].std(), "2xStdDev": (ratio_statistic_finder[column].std()*2)  }}) 

ratio_statistic_dataframe = pd.DataFrame.from_dict(ratio_statistic_data)
ratio_statistic_dataframe = ratio_statistic_dataframe.transpose()

updated_ticks = estrategies.update_quotes(just_last_update=True,mode=InformationType.Real_Time)
ticks_to_process = updated_ticks[0]
options_updated = []
for i in ticks_to_process:
    options_updated.append(i.__dict__)

df_options_updated = convert_dict_from_update_ticks_to_dataframe(options_updated)
df_options_updated.reset_index()
df_options_updated = df_options_updated.set_index('option_name')
df_options_updated = pd.merge(df_options_updated, ratio_statistic_dataframe, left_index=True, right_index=True)

df_options_updated['above_mean'] = abs(df_options_updated['ratio']) > abs(df_options_updated['Mean'])
df_options_updated['above_2x_std_dev'] = abs(df_options_updated['ratio']) > abs(abs(df_options_updated['Mean']) + df_options_updated['2xStdDev'])
df_options_updated

,strike,bid,ask,last_tick,ratio,stock_price,deal_type_zone,updated_at,Mean,StdDev,2xStdDev,above_mean,above_2x_std_dev
PETRH269,26.21,2.06,2.08,2.06,-0.16,27.5,ITM,2021-07-15 11:13:01.893,1.485424,0.861697,1.723394,False,False
PETRH254,26.46,1.84,1.90,1.89,-0.17,27.5,ITM,2021-07-15 11:13:01.893,-0.188293,0.030892,0.061783,False,False
PETRH274,26.71,1.71,1.74,1.71,-0.18,27.5,ITM,2021-07-15 11:13:01.893,-0.170757,0.031322,0.062644,True,False
PETRH276,26.96,1.55,1.59,1.57,-0.14,27.5,ITM,2021-07-15 11:13:01.893,-0.156187,0.030873,0.061747,False,False
PETRH261,27.21,1.42,1.44,1.41,-0.16,27.5,ITM,2021-07-15 11:13:01.893,-0.160704,0.019482,0.038964,False,False
PETRH247,27.96,1.03,1.05,1.03,-0.12,27.5,OTM,2021-07-15 11:13:01.893,-0.137712,0.025105,0.050209,False,False
PETRH289,28.21,0.92,0.93,0.92,-0.11,27.5,OTM,2021-07-15 11:13:01.893,-0.116907,0.011717,0.023433,False,False
PETRH279,28.46,0.82,0.84,0.83,-0.09,27.5,OTM,2021-07-15 11:13:01.893,-0.113552,0.011530,0.023061,False,False
PETRH294,28.71,0.73,0.74,0.73,-0.10,27.5,OTM,2021-07-15 11:13:01.893,-0.102437,0.011202,0.022404,False,False
PETRH243,28.96,0.64,0.66,0.65,-0.08,27.5,OTM,2021-07-15 11:13:01.893,-0.093312,0.010704,0.021409,False,False
